In [1]:
import pandas as pd
import numpy as np

In [2]:
contests = pd.read_csv('data/WorkingData.csv')[:10000]
print("There are " + str(len(contests)) + " Contests")

There are 10000 Contests


In [3]:
print(contests.columns)

Index(['Unnamed: 0', 'ContestId', 'SportName', 'VariantName', 'GameSet',
       'ContestName', 'ContestStartDatetimeEST', 'ContestEndDatetimeEST',
       'ContestPayoutDatetimeEST', 'EntryFeeAmount', 'TotalPrizeAmount',
       'MaxNumberPlayers', 'MaxEntriesPerUser', 'Entries', 'DistinctUsers',
       'Contest_Group', 'NumGames', 'DraftablePlayersInSet',
       'PaidUsersInDraftGroup', 'TopPrize', 'MaxPayoutPosition', 'is_vault'],
      dtype='object')


## We need:
Add threshold, Number of entries 4 hours out etc. 


In [4]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

In [5]:
# Get rid of cheats
features = contests.columns.drop('Entries').drop('DistinctUsers')
# Get rid of Useless 
#drop('ContestId')
features = features.drop('Unnamed: 0').drop('GameSet').drop('ContestName').drop('is_vault')
contests = contests[features].dropna()

In [6]:
contests.tail()

,ContestId,SportName,VariantName,ContestStartDatetimeEST,ContestEndDatetimeEST,ContestPayoutDatetimeEST,EntryFeeAmount,TotalPrizeAmount,MaxNumberPlayers,MaxEntriesPerUser,Contest_Group,NumGames,DraftablePlayersInSet,PaidUsersInDraftGroup,TopPrize,MaxPayoutPosition
9995,13002622,SOC,Classic,2015-10-31 11:00:00.000,2015-10-31 11:00:00.000,2015-10-31 16:16:00.000,2.00,750.0,430,10,Headliner,6,330.0,5764,150.0,100
9996,11978260,CFB,Classic,2015-10-17 20:00:00.000,2015-10-17 22:30:00.000,2015-10-18 03:46:15.000,2.00,150.0,85,8,Headliner,5,411.0,10579,30.0,15
9997,11984211,CFB,Classic,2015-10-17 22:00:00.000,2015-10-17 22:30:00.000,2015-10-18 03:51:23.000,27.00,2000.0,84,5,Headliner,3,247.0,12968,400.0,20
9998,12019074,NAS,Classic,2015-10-18 14:15:00.000,2015-10-18 14:15:00.000,2015-10-18 18:11:08.000,5.00,1500.0,345,10,Headliner,1,44.0,41291,175.0,75
9999,12019257,NBA,Classic,2015-10-18 13:00:00.000,2015-10-18 21:00:00.000,2015-10-18 23:50:02.000,0.25,50.0,235,10,Headliner,7,267.0,4861,10.0,50


### Categorizing

In [7]:
def categorizeFeature(df, name):
    return df.assign(newCol=lb_make.fit_transform(categoricalDF[name])).rename(columns={'newCol': name+'_code'})

In [8]:
categoricalFeatures = ['SportName', 'VariantName', 'Contest_Group']
categoricalDF = contests[['ContestId', 'SportName', 'VariantName', 'Contest_Group']]
for f in categoricalFeatures:
    categoricalDF = categorizeFeature(categoricalDF, f)
categoricalDF = categoricalDF.drop(categoricalFeatures, axis=1)

categoricalDF.tail()

,ContestId,SportName_code,VariantName_code,Contest_Group_code
9995,13002622,9,0,5
9996,11978260,0,0,5
9997,11984211,0,0,5
9998,12019074,4,0,5
9999,12019257,5,0,5


### Scaling

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [10]:
def scaleFeature(df, name):
    return df.assign(newCol=scaler.fit_transform(df[[name]])).rename(columns={'newCol': name+'_Scaled'})

In [11]:
scaleFeatures = ['EntryFeeAmount', 'TotalPrizeAmount', 'MaxNumberPlayers', 'MaxEntriesPerUser', 
                       'NumGames', 'DraftablePlayersInSet', 'PaidUsersInDraftGroup', 'TopPrize', 'MaxPayoutPosition']
scaleDF = contests[['ContestId', 'EntryFeeAmount', 'TotalPrizeAmount', 'MaxNumberPlayers', 'MaxEntriesPerUser', 
                       'NumGames', 'DraftablePlayersInSet', 'PaidUsersInDraftGroup', 'TopPrize', 'MaxPayoutPosition']]
for f in scaleFeatures:
    scaleDF = scaleFeature(scaleDF, f)
scaleDF = scaleDF.drop(scaleFeatures, axis=1)
scaleDF.tail()

,ContestId,EntryFeeAmount_Scaled,TotalPrizeAmount_Scaled,MaxNumberPlayers_Scaled,MaxEntriesPerUser_Scaled,NumGames_Scaled,DraftablePlayersInSet_Scaled,PaidUsersInDraftGroup_Scaled,TopPrize_Scaled,MaxPayoutPosition_Scaled
9995,13002622,0.000063,7.050032e-05,0.000744,9.000000e-09,0.277778,0.201954,0.006823,0.000074,0.000788
9996,11978260,0.000063,1.050005e-05,0.000144,7.000000e-09,0.222222,0.254723,0.013252,0.000014,0.000111
9997,11984211,0.000955,1.955009e-04,0.000143,4.000000e-09,0.111111,0.147883,0.016442,0.000199,0.000151
9998,12019074,0.000170,1.455007e-04,0.000597,9.000000e-09,0.000000,0.015635,0.054261,0.000087,0.000589
9999,12019257,0.000000,5.000023e-07,0.000405,9.000000e-09,0.333333,0.160912,0.005617,0.000004,0.000390


### Time
time data : month, day of month, day of week, hours


In [12]:
timeFeatures = ['ContestId', 'ContestStartDatetimeEST', 'ContestEndDatetimeEST', 'ContestPayoutDatetimeEST']
mapper = {'ContestStartDatetimeEST' : 'Start', 'ContestEndDatetimeEST' : 'End', 'ContestPayoutDatetimeEST': 'Payout'}
timeDF = contests[timeFeatures]
formatString = '%Y-%m-%d %H:%M:%S'

In [13]:
timeDF.tail()

,ContestId,ContestStartDatetimeEST,ContestEndDatetimeEST,ContestPayoutDatetimeEST
9995,13002622,2015-10-31 11:00:00.000,2015-10-31 11:00:00.000,2015-10-31 16:16:00.000
9996,11978260,2015-10-17 20:00:00.000,2015-10-17 22:30:00.000,2015-10-18 03:46:15.000
9997,11984211,2015-10-17 22:00:00.000,2015-10-17 22:30:00.000,2015-10-18 03:51:23.000
9998,12019074,2015-10-18 14:15:00.000,2015-10-18 14:15:00.000,2015-10-18 18:11:08.000
9999,12019257,2015-10-18 13:00:00.000,2015-10-18 21:00:00.000,2015-10-18 23:50:02.000


In [32]:
def mapTime(df, name, colHeader):
    newDF = df[[name]]
    newDF = newDF.assign(timeCol = lambda x: pd.to_datetime(df[name], format=formatString))
    newDF[colHeader+'_Minute'] = newDF.timeCol.apply(lambda x: x.strftime('%M'))
    newDF[colHeader+'_Hour'] = newDF.timeCol.apply(lambda x: x.strftime('%H'))
    newDF[colHeader+'_Day'] = newDF.timeCol.apply(lambda x: x.strftime('%d'))
    newDF[colHeader+'_Month'] = newDF.timeCol.apply(lambda x: x.strftime('%m'))
    newDF[colHeader+'_Year'] = newDF.timeCol.apply(lambda x: x.strftime('%Y'))
    newDF[colHeader+'_DayOfWeek'] = newDF.timeCol.apply(lambda x: x.strftime('%w'))
    newDF[colHeader+'_DayOfYear'] = newDF.timeCol.apply(lambda x: x.strftime('%-j'))
    newDF[colHeader+'_WeekOfYear'] = newDF.timeCol.apply(lambda x: x.strftime('%U'))
    newDF = newDF.drop(columns = [name, 'timeCol'], axis=1)
    return newDF

This next one takes ages, hence the 'timeDFBackup=timesDF'

In [15]:
timesDF = timeDF[['ContestId']]
timeDF.head()
for f in mapper:
    timesDF = pd.merge(timesDF, mapTime(timeDF, f, mapper[f]), on="ContestId").drop(columns=[f], axis=1)
timeDFBackup=timesDF

### Scale Time Data

In [16]:
timesDF.tail()

,ContestId,Start_Minute,Start_Hour,Start_Day,Start_Month,Start_Year,Start_DayOfWeek,Start_DayOfYear,Start_WeekOfYear,End_Minute,...,End_DayOfYear,End_WeekOfYear,Payout_Minute,Payout_Hour,Payout_Day,Payout_Month,Payout_Year,Payout_DayOfWeek,Payout_DayOfYear,Payout_WeekOfYear
9995,13002622,0,11,31,10,2015,6,304,43,0,...,304,43,16,16,31,10,2015,6,304,43
9996,11978260,0,20,17,10,2015,6,290,41,30,...,290,41,46,3,18,10,2015,0,291,42
9997,11984211,0,22,17,10,2015,6,290,41,30,...,290,41,51,3,18,10,2015,0,291,42
9998,12019074,15,14,18,10,2015,0,291,42,15,...,291,42,11,18,18,10,2015,0,291,42
9999,12019257,0,13,18,10,2015,0,291,42,0,...,291,42,50,23,18,10,2015,0,291,42


In [17]:
scaleTimeFeatures = timesDF.columns.drop('ContestId')
scaleTimeFeatures

Index(['Start_Minute', 'Start_Hour', 'Start_Day', 'Start_Month', 'Start_Year',
       'Start_DayOfWeek', 'Start_DayOfYear', 'Start_WeekOfYear', 'End_Minute',
       'End_Hour', 'End_Day', 'End_Month', 'End_Year', 'End_DayOfWeek',
       'End_DayOfYear', 'End_WeekOfYear', 'Payout_Minute', 'Payout_Hour',
       'Payout_Day', 'Payout_Month', 'Payout_Year', 'Payout_DayOfWeek',
       'Payout_DayOfYear', 'Payout_WeekOfYear'],
      dtype='object')

In [18]:
for f in scaleTimeFeatures:
    timesDF = scaleFeature(timesDF, f)
timesDF = timesDF.drop(scaleTimeFeatures, axis=1)

In [19]:
timesDF.tail()

,ContestId,Start_Minute_Scaled,Start_Hour_Scaled,Start_Day_Scaled,Start_Month_Scaled,Start_Year_Scaled,Start_DayOfWeek_Scaled,Start_DayOfYear_Scaled,Start_WeekOfYear_Scaled,End_Minute_Scaled,...,End_DayOfYear_Scaled,End_WeekOfYear_Scaled,Payout_Minute_Scaled,Payout_Hour_Scaled,Payout_Day_Scaled,Payout_Month_Scaled,Payout_Year_Scaled,Payout_DayOfWeek_Scaled,Payout_DayOfYear_Scaled,Payout_WeekOfYear_Scaled
9995,13002622,0.000000,0.388889,1.000000,0.5,0.0,1.0,0.980769,0.875,0.000000,...,0.962264,0.875,0.271186,0.695652,1.000000,0.5,0.0,1.0,0.944444,0.875
9996,11978260,0.000000,0.888889,0.533333,0.5,0.0,1.0,0.711538,0.625,0.545455,...,0.698113,0.625,0.779661,0.130435,0.566667,0.5,0.0,0.0,0.703704,0.750
9997,11984211,0.000000,1.000000,0.533333,0.5,0.0,1.0,0.711538,0.625,0.545455,...,0.698113,0.625,0.864407,0.130435,0.566667,0.5,0.0,0.0,0.703704,0.750
9998,12019074,0.272727,0.555556,0.566667,0.5,0.0,0.0,0.730769,0.750,0.272727,...,0.716981,0.750,0.186441,0.782609,0.566667,0.5,0.0,0.0,0.703704,0.750
9999,12019257,0.000000,0.500000,0.566667,0.5,0.0,0.0,0.730769,0.750,0.000000,...,0.716981,0.750,0.847458,1.000000,0.566667,0.5,0.0,0.0,0.703704,0.750


### Splitting Scaling into 2 columns

In [20]:
import math

In [21]:
def circularizeFeature(df, feature):
    fStr = feature[:-len('scaled')]
    df = df.assign(X= lambda c: np.sin(df[feature]*2*math.pi)).assign(Y= lambda c: np.cos(df[feature]*2*math.pi))
    df = df.rename(columns={'X': fStr+'X', 'Y': fStr+'Y'})
    return df

In [22]:
circularFeatures = timesDF.columns
circleDF = timesDF[circularFeatures]
for f in circularFeatures:
    if f != 'ContestId':
        circleDF = circularizeFeature(circleDF, f)
circleDF = circleDF.drop(circularFeatures.drop('ContestId'), axis=1)

In [23]:
circleDF.tail()

,ContestId,Start_Minute_X,Start_Minute_Y,Start_Hour_X,Start_Hour_Y,Start_Day_X,Start_Day_Y,Start_Month_X,Start_Month_Y,Start_Year_X,...,Payout_Month_X,Payout_Month_Y,Payout_Year_X,Payout_Year_Y,Payout_DayOfWeek_X,Payout_DayOfWeek_Y,Payout_DayOfYear_X,Payout_DayOfYear_Y,Payout_WeekOfYear_X,Payout_WeekOfYear_Y
9995,13002622,0.000000,1.000000,6.427876e-01,-0.766044,-1.133108e-15,1.000000,1.224647e-16,-1.0,0.0,...,1.224647e-16,-1.0,0.0,1.0,-2.449294e-16,1.0,-0.34202,0.939693,-0.707107,7.071068e-01
9996,11978260,0.000000,1.000000,-6.427876e-01,0.766044,-2.079117e-01,-0.978148,1.224647e-16,-1.0,0.0,...,1.224647e-16,-1.0,0.0,1.0,0.000000e+00,1.0,-0.95799,-0.286803,-1.000000,-1.836970e-16
9997,11984211,0.000000,1.000000,-1.133108e-15,1.000000,-2.079117e-01,-0.978148,1.224647e-16,-1.0,0.0,...,1.224647e-16,-1.0,0.0,1.0,0.000000e+00,1.0,-0.95799,-0.286803,-1.000000,-1.836970e-16
9998,12019074,0.989821,-0.142315,-3.420201e-01,-0.939693,-4.067366e-01,-0.913545,1.224647e-16,-1.0,0.0,...,1.224647e-16,-1.0,0.0,1.0,0.000000e+00,1.0,-0.95799,-0.286803,-1.000000,-1.836970e-16
9999,12019257,0.000000,1.000000,1.224647e-16,-1.000000,-4.067366e-01,-0.913545,1.224647e-16,-1.0,0.0,...,1.224647e-16,-1.0,0.0,1.0,0.000000e+00,1.0,-0.95799,-0.286803,-1.000000,-1.836970e-16


### Calculating threshold

In [24]:
calcFeatures = ['ContestId', 'TotalPrizeAmount','EntryFeeAmount']
calcDF = contests[calcFeatures]
calcDF = calcDF.assign(SuccessThreshold= lambda x:(calcDF['TotalPrizeAmount'].astype(float)/(calcDF['EntryFeeAmount'].astype(float))).astype(int))
calcDF = calcDF[['ContestId', 'SuccessThreshold']]

In [25]:
calcDF = scaleFeature(calcDF, 'SuccessThreshold')
threshDF = calcDF[['ContestId','SuccessThreshold']]
scaledThreshDF = calcDF[['ContestId','SuccessThreshold_Scaled']]
# calcDF
# calcDF = calcDF.drop(columns=['SuccessThreshold'], axis=1)

In [26]:
calcDF.tail()

,ContestId,SuccessThreshold,SuccessThreshold_Scaled
9995,13002622,375,0.000748
9996,11978260,75,0.000148
9997,11984211,74,0.000146
9998,12019074,300,0.000598
9999,12019257,200,0.000398


### Calculating Time-Series Data

In [27]:
seriesDF = contests[['ContestId', ]]
cIds = seriesDF['ContestId']
cTimeDF = timeDFBackup[['ContestId','Start_Month', 'Start_Year', 'End_Month', 'End_Year']]
seekHours = [0, 1, 2, 3, 4, 6, 8]
# seekHours = [0, 1, 2, 3, 4, 5, 6, 8, 10, 12, 16, 20, 24, 36, 48]

In [28]:
def seriesByMonthAndYear(year, month):
    seriesDF = pd.read_csv('data/WPI/'+ str(year) + '-' + str(month) + '.csv', header=None)
    seriesDF.columns = ['ContestId', 'SecondsRemaining', 'Entries']
    return seriesDF

In [29]:
#times should be an array of hours in ascending order i.e.[2, 4, 6, 8]
def getTimeSplits(df, times):
    entrySums = []
    for hour in times:
        entrySums.append(df[(df['SecondsRemaining'] >= hour*60*60)].Entries.sum())
    return entrySums

In [30]:
def getTimeSeriesData(ContestId):
    contestDF = cTimeDF.loc[cTimeDF['ContestId']==ContestId]
    
    sMonth = (contestDF.Start_Month).item()
    if(sMonth <= 9):
        sMonth = "0"+str(sMonth)
    sYear = (contestDF.Start_Year).item()
    eMonth = (contestDF.End_Month).item()
    if(eMonth <= 9):
        eMonth = "0"+str(eMonth)
    eYear = (contestDF.End_Year).item()
    
    seriesDF = seriesByMonthAndYear(sYear, sMonth)
    if sMonth != eMonth:
        seriesDF = seriesDF.append(seriesByMonthAndYear( eYear, eMonth))
    seriesDF = seriesDF.loc[seriesDF['ContestId']==ContestId]
    timeSplits = getTimeSplits(seriesDF, seekHours)
    hourData = {'ContestId':ContestId}
    for i in range(len(seekHours)):
        hours = seekHours[i]
        entries = timeSplits[i]
        hourData[str(hours)+"HoursOut"] = [entries]
    newSeriesDF = pd.DataFrame(data=hourData)
    return newSeriesDF

In [33]:
seriesDF = pd.DataFrame()
for cId in cIds:
    seriesDF = seriesDF.append(getTimeSeriesData(cId), ignore_index=True)

### Scale Series to Threshold

In [ ]:
scaledSeriesDF = pd.merge(threshDF, seriesDF, on="ContestId")

In [ ]:
scaledSeriesDF.tail()

In [ ]:
def percentifyFeature(df, numerator, denomenator):
    return df.assign(newCol=lambda x: df[numerator].astype(int)/df[denomenator].astype(int)).rename(columns={'newCol': numerator+'_Percent'})

In [ ]:
percentifyFeatures = seriesDF.columns.drop('ContestId')

In [ ]:
for f in percentifyFeatures:
    scaledSeriesDF = percentifyFeature(scaledSeriesDF, f, 'SuccessThreshold')
scaledSeriesDF = scaledSeriesDF.drop(seriesDF.columns.drop('ContestId'), axis=1).drop(['SuccessThreshold'], axis=1)

### Merge altogether

# RANDOM THOUGHT- LETS DETERMINE WHEN MOST CONTESTS REACH THRESHOLD

In [ ]:
# mergable = [categoricalDF, scaleDF, circleDF, scaledThreshDF, scaledSeriesDF]
mergable = [categoricalDF, scaleDF, circleDF, scaledThreshDF]

In [ ]:
mergedDF = contests[['ContestId']]
for df in mergable:
    mergedDF = pd.merge(mergedDF, df, on='ContestId')

In [ ]:
mergedDF.tail()

In [ ]:
mergedDF.to_csv("data/RandomForest_Features_notime.csv")

In [ ]:
len(mergedDF)